# ZTF alerts triggered in TESS Northern Fields

The [Zwicky Transient Facility (ZTF)](https://ztf.caltech.edu) will conduct a public survey of all 13 TESS northern sectors in 2019-2020. The sectors (as much as is visible from the Palomar Observatory on a particular night) will be mapped every night in the $g$ and $r$ filters concurrently with the TESS observations. 

The observations of the first TESS Northern sector (sector 14) have begun on July 18, 2019. 

The ZTF alerts for the transient events discovered in the TESS fields will have the [alert packet](https://zwickytransientfacility.github.io/ztf-avro-alert/) field `programpi` in the `candidate` block set to `TESS`. Note that events may be triggered from transient, reoccurring variable, or moving objects.

A compressed nightly tar-ball with the alert packets converted into the `JSON` format will be immediately made available to the public via ZTF's bucket on `Google Cloud`. The alerts in the TESS fields will be additionally cross-matched against a number of external catalogs. Furthermore, we anticipate executing multiple machine learning (ML) classifiers on the alerts. Initially, the catalogs will include `2MASS_PSC`, `AllWISE`, `IPHAS_DR2`, and `Gaia_DR2` and the ML classifiers will include `braai`, a deep-learning-based real-bogus classifier.

## Accessing the alerts

### Downloading the data

If you have a `Google` account (any), you can access the nightly tar-balls at [https://console.cloud.google.com/storage/browser/ztf-tess](https://console.cloud.google.com/storage/browser/ztf-tess)

Alternatively, individual tar-balls may be downloaded at https://storage.googleapis.com/ztf-tess/YYYYMMDD.tar.gz

You can also list and download the data programmatically.

In [36]:
from IPython.core.display import display, HTML
!pip install requests
import requests

In [35]:
# a library to print large numbers in human-readable format
!pip install humanize
import humanize

Get a list of individual tar-ball meta-data:

In [37]:
r = requests.get('https://www.googleapis.com/storage/v1/b/ztf-tess/o')
tbs = r.json()['items']

In [39]:
for tb in tbs:
    display(HTML(f"""<a href="{tb['mediaLink']}">{tb['name']}</a> (size: {humanize.naturalsize(tb['size'])})"""))

Let's download and unpack the data for July 14, 2019.

In [47]:
#!wget https://storage.googleapis.com/ztf-tess/20190714.tar.gz
#!tar -xf 20190714.tar.gz

### Displaying the data

Let us load, parse, and display a few alerts.

An alert from a know SN:

In [ ]:
candid = 924463071815015006

with open(f'20190714/{candid}.json', 'r') as 